In [1]:
import sys
sys.path.append('../')
import nbimporter

from _04_Heating_and_Cooling_System import _04_07_Hot_water_floor_heating as _04_07_MainPart
from _04_Heating_and_Cooling_System import _04_07_AppB_Gas_Boiler as _04_07_AppB_gas
from _04_Heating_and_Cooling_System import _04_07_AppA_Kerosene_Boiler as _04_07_AppA_ker
from _07_Hot_Water_Supply_System import _07_AppC_Gas_Boiler as _07_AppC_gas
from _07_Hot_Water_Supply_System import _07_AppD_Kerosene_Boiler as _07_AppD_ker

import numpy as np
from enum import Enum

Importing Jupyter notebook from ..\_04_Heating_and_Cooling_System\_04_07_Hot_water_floor_heating.ipynb
Importing Jupyter notebook from ..\_04_Heating_and_Cooling_System\_04_07_AppB_Gas_Boiler.ipynb
Importing Jupyter notebook from ..\_04_Heating_and_Cooling_System\_04_07_AppA_Kerosene_Boiler.ipynb
Importing Jupyter notebook from ..\_07_Hot_Water_Supply_System\_07_AppC_Gas_Boiler.ipynb
Importing Jupyter notebook from ..\_07_Hot_Water_Supply_System\_07_AppD_Kerosene_Boiler.ipynb


# Chapter VIII Cogeneration System / コージェネレーション設備

## 1. Introduction

This calculation depends on the specification of "Chapter VIII Cogeneration System" in BRI

In [2]:
def CGS_Spec(_CGS_name):
    spec = {'GEC1'  : {'ExaustUseForWaterHeating' : 'Yes', 'ExaustUsePriority' : 'HotWater', 'BB_HW' : {'Type' : 'gas', 'Ita' : 0.782}, 'BB_WH' : {'Type' : 'gasConv'      , 'Ita' : 0.82, 'RatedPower' : 17400}, 'PU_exaust_rate' : {'HotWater' : 0.7494, 'WaterHeating' : 0.7758}, 'PU_GenerationMethod' : 'HeatPriority'            , 'PU_virtualGeneration' : {'a_PU' : 0.0000, 'a_DHW' : 0.1398, 'a_HWH' : 0.1398, 'b' :   7.7827, 'c' : 0.6925}, 'PU_virtualFuel' : {'a_DHW' : 1, 'a_HWH' : 0}, 'PU_UpperCoef' : {'a_DHW' : -0.0075, 'a_HWH' : -0.0075, 'b' : 1.4847}, 'PU_GenIta' : {'a_PU' : 0.000000, 'a_DHW' : -0.000034, 'a_HWH' : -0.000034, 'b' : 0.1779, 'UpperLimit' : 0.2069, 'LowerLimit' : 0.1499}, 'PU_ExaustIta' : {'a_PU' : 0.000000, 'a_DHW' : 0.000342, 'a_HWH' : 0.000342, 'b' : 0.5787, 'UpperLimit' : 0.5787, 'LowerLimit' : 0.5576}, 'RatedPower' : 1000, 'TU_auxPower' :{'HotWater' : 15.1, 'WaterHeating' : 132.1}},
            'PEFC5' : {'ExaustUseForWaterHeating' : 'Yes', 'ExaustUsePriority' : 'HotWater', 'BB_HW' : {'Type' : 'gas', 'Ita' : 0.782}, 'BB_WH' : {'Type' : 'gasConv'      , 'Ita' : 0.82, 'RatedPower' : 17400}, 'PU_exaust_rate' : {'HotWater' : 0.9654, 'WaterHeating' : 0.0000}, 'PU_GenerationMethod' : 'HeatPriority'            , 'PU_virtualGeneration' : {'a_PU' : 1.2469, 'a_DHW' : 0.0000, 'a_HWH' : 0.0000, 'b' : -14.9213, 'c' : 0.9950}, 'PU_virtualFuel' : {'a_DHW' : 1, 'a_HWH' : 0}, 'PU_UpperCoef' : {'a_DHW' :  0.0000, 'a_HWH' :  0.0000, 'b' : 1.0526}, 'PU_GenIta' : {'a_PU' : 0.003096, 'a_DHW' :  0.000000, 'a_HWH' :  0.000000, 'b' : 0.1483, 'UpperLimit' : 0.3193, 'LowerLimit' : 0.2301}, 'PU_ExaustIta' : {'a_PU' : 0.000000, 'a_DHW' : 0.000452, 'a_HWH' : 0.000000, 'b' : 0.3017, 'UpperLimit' : 0.3716, 'LowerLimit' : 0.2803}, 'RatedPower' : 700 , 'TU_auxPower' :{'HotWater' : 15.4, 'WaterHeating' : 138.1}},
            'SOFC1' : {'ExaustUseForWaterHeating' : 'No', 'ExaustUsePriority' : 'N/A'      , 'BB_HW' : {'Type' : 'gas', 'Ita' : 0.905}, 'BB_WH' : {'Type' : 'gasCondensing', 'Ita' : 0.87, 'RatedPower' : 17400}, 'PU_exaust_rate' : {'HotWater' : 0.7227, 'WaterHeating' : 0.0000}, 'PU_GenerationMethod' : 'ElectricityPriority'     , 'PU_virtualGeneration' : {'a_PU' : 1.1175, 'a_DHW' : 0.0000, 'a_HWH' : 0.0000, 'b' :  -6.6385, 'c' : 0.9950}, 'PU_virtualFuel' : {'a_DHW' : 0, 'a_HWH' : 0}, 'PU_UpperCoef' : {'a_DHW' :  0     , 'a_HWH' :  0     , 'b' : 0     }, 'PU_GenIta' : {'a_PU' : 0.003600, 'a_DHW' :  0.000000, 'a_HWH' :  0.000000, 'b' : 0.1951, 'UpperLimit' : 0.3925, 'LowerLimit' : 0.3092}, 'PU_ExaustIta' : {'a_PU' : 0.000500, 'a_DHW' : 0.000000, 'a_HWH' : 0.000000, 'b' : 0.3135, 'UpperLimit' : 0.3539, 'LowerLimit' : 0.3169}, 'RatedPower' : 700 , 'TU_auxPower' :{'HotWater' : 17.1, 'WaterHeating' : 0    }},
            'SOFC2' : {'ExaustUseForWaterHeating' : 'No' , 'ExaustUsePriority' : 'N/A'     , 'BB_HW' : {'Type' : 'gas', 'Ita' : 0.905}, 'BB_WH' : {'Type' : 'gasCondensing', 'Ita' : 0.87, 'RatedPower' : 17400}, 'PU_exaust_rate' : {'HotWater' : 0.6885, 'WaterHeating' : 0.0000}, 'PU_GenerationMethod' : 'ElectricityPriority'     , 'PU_virtualGeneration' : {'a_PU' : 1.1262, 'a_DHW' : 0.0000, 'a_HWH' : 0.0000, 'b' :  -6.5572, 'c' : 0.9846}, 'PU_virtualFuel' : {'a_DHW' : 0, 'a_HWH' : 0}, 'PU_UpperCoef' : {'a_DHW' :  0     , 'a_HWH' :  0     , 'b' : 0     }, 'PU_GenIta' : {'a_PU' : 0.005800, 'a_DHW' :  0.000000, 'a_HWH' :  0.000000, 'b' : 0.1611, 'UpperLimit' : 0.4290, 'LowerLimit' : 0.3503}, 'PU_ExaustIta' : {'a_PU' : 0.002800, 'a_DHW' : 0.000000, 'a_HWH' : 0.000000, 'b' : 0.1854, 'UpperLimit' : 0.3179, 'LowerLimit' : 0.2756}, 'RatedPower' : 700 , 'TU_auxPower' :{'HotWater' : 11.8, 'WaterHeating' : 0    }}
    }[_CGS_name]
    return spec
    

In [3]:
CGS_Spec('SOFC2')

{'BB_HW': {'Ita': 0.905, 'Type': 'gas'},
 'BB_WH': {'Ita': 0.87, 'RatedPower': 17400, 'Type': 'gasCondensing'},
 'ExaustUseForWaterHeating': 'No',
 'ExaustUsePriority': 'N/A',
 'PU_ExaustIta': {'LowerLimit': 0.2756,
  'UpperLimit': 0.3179,
  'a_DHW': 0.0,
  'a_HWH': 0.0,
  'a_PU': 0.0028,
  'b': 0.1854},
 'PU_GenIta': {'LowerLimit': 0.3503,
  'UpperLimit': 0.429,
  'a_DHW': 0.0,
  'a_HWH': 0.0,
  'a_PU': 0.0058,
  'b': 0.1611},
 'PU_GenerationMethod': 'ElectricityPriority',
 'PU_UpperCoef': {'a_DHW': 0, 'a_HWH': 0, 'b': 0},
 'PU_exaust_rate': {'HotWater': 0.6885, 'WaterHeating': 0.0},
 'PU_virtualFuel': {'a_DHW': 0, 'a_HWH': 0},
 'PU_virtualGeneration': {'a_DHW': 0.0,
  'a_HWH': 0.0,
  'a_PU': 1.1262,
  'b': -6.5572,
  'c': 0.9846},
 'RatedPower': 700,
 'TU_auxPower': {'HotWater': 11.8, 'WaterHeating': 0}}

## 5. Gas Consumption and Kerosene Consumption / ガス消費量及び灯油消費量

$$\Large
E_{CG} = E_{F,PU} + E_{F,BB,DHW} + E_{F,BB,HWH}
\tag{1}
$$

Where  
$E_{CG}$ is the daily primary energy consumption of cogeneration system / 1日当たりのコージェネレーション設備の一次エネルギー消費量 ( MJ/d );  
$E_{F,PU}$ is the daily fuel consumption of power generation unit / 1日当たりの発電ユニットの燃料消費量 ( MJ/d );  
$E_{F,BB,DHW}$ is the daily fuel consumption of backup boiler during hot water supply / 1日当たりの給湯時のバックアップボイラーの燃料消費量 ( MJ/d );  
$E_{F,BB,HWH}$ is the daily fuel consumption of backup boiler during hot water heating / 1日当たりの温水暖房時のバックアップボイラーの燃料消費量 ( MJ/d ).  

In [4]:
# 式(1)
get_E_CG = lambda _E_F_PU, _E_F_BB_DHW, _E_F_BB_HWH : _E_F_PU + _E_F_BB_DHW + _E_F_BB_HWH

## 6. Amount of Power Generation / 発電量

$$\Large
E_{E,CG,gen,t} = E_{E,gen,PU,t} - E_{E,TU,aux,t}
\tag{2}
$$


Where  
$E_{E,CG,gen,t}$ is the hourly power generation by cogeneration system / 1時間当たりのコージェネレーション設備による発電量 ( kWh/h );  
$E_{E,gen,PU,t}$ is the hourly power generation of power generation unit / 1時間当たりの発電ユニットの発電量 ( kWh/h );  
$E_{E,TU,aux,t}$ is the hourly auxiliary machine power consumption of the tank unit / 1時間当たりのタンクユニットの補機消費電力量 ( kWh/h ).  

In [5]:
# 式(2)
get_E_E_CG_gen_t = lambda _E_E_gen_PU_t, _E_E_TU_aux_t : _E_E_gen_PU_t - _E_E_TU_aux_t

#### Example

In [6]:
get_E_E_CG_gen_t(10,8.5)

1.5

## 7.Annual Energy Efficiency of Buckup Boiler of Hot Water Supply / 給湯時のバックアップボイラーの年間平均効率

$$\Large
e_{BB,ave} = \frac {\sum^{365}_{d=1} L_{BB,DHW}} {\sum^{365}_{d=1} E_{F,BB,DHW}}
\tag{3}
$$

In [7]:
# 式(3)
get_e_BB_ave = lambda _L_BB_DHW, _E_F_BB_DHW : np.sum(_L_BB_DHW) / np.sum(_E_F_BB_DHW)

#### Example

In [8]:
_L_BB_DHW = (5,10,15)
_E_F_BB_DHW = (10,20,30)
get_e_BB_ave(_L_BB_DHW, _E_F_BB_DHW)

0.5

## 8. Fuel Consumption of Backup Boiler of Hot Water Supply / 給湯時のバックアップボイラーの燃料消費量

### 8.1 Fuel Consumption / 燃料消費量

In [9]:
# 給湯設備　付録C、D
def get_E_hs( _CGS_name, _theta_ex_ave, _L_2dash_k, _L_2dash_s, _L_2dash_w, _L_2dash_b1, _L_2dash_b2, _L_2dash_ba1, _L_2dash_ba2 ):
    _Type = CGS_Spec(_CGS_name)['BB_HW']['Type']
    _Ita = CGS_Spec(_CGS_name)['BB_HW']['Ita']
    
    if _Type == 'gas':
        f_hs = ( 0.8754 * _Ita + 0.060 ) / 0.745
        a_k,   b_k,   c_k   = 0.0019 * f_hs,  0.0013 * f_hs, 0.6533 * f_hs
        a_s,   b_s,   c_s   = 0.0006 * f_hs,  0.0005 * f_hs, 0.7414 * f_hs
        a_w,   b_w,   c_w   = 0.0019 * f_hs,  0.0013 * f_hs, 0.6533 * f_hs
        a_b1,  b_b1,  c_b1  = 0.0000 * f_hs,  0.0002 * f_hs, 0.7839 * f_hs
        a_b2,  b_b2,  c_b2  = 0.0000 * f_hs, -0.0005 * f_hs, 0.7828 * f_hs
        a_ba1, b_ba1, c_ba1 = 0.0000 * f_hs,  0.0002 * f_hs, 0.7839 * f_hs
        a_ba2, b_ba2, c_ba2 = 0.0033 * f_hs,  0.0194 * f_hs, 0.5776 * f_hs
        e_k   = min( a_k   * _theta_ex_ave + b_k   * ( L_2dash_k + L_2dash_w ) + c_k,   1.0 )
        e_s   = min( a_s   * _theta_ex_ave + b_s   *   L_2dash_s               + c_s,   1.0 )
        e_w   = min( a_w   * _theta_ex_ave + b_w   * ( L_2dash_k + L_2dash_w ) + c_w,   1.0 )
        e_b1  = min( a_b1  * _theta_ex_ave + b_b1  *   L_2dash_b1              + c_b1,  1.0 )
        e_b2  = min( a_b2  * _theta_ex_ave + b_b2  *   L_2dash_b2              + c_b2,  1.0 )
        e_ba1 = min( a_ba1 * _theta_ex_ave + b_ba1 *   L_2dash_ba1             + c_ba1, 1.0 )
        e_ba2 = min( a_ba2 * _theta_ex_ave + b_ba2 *   L_2dash_ba2             + c_ba2, 1.0 )
    elif _Type == 'kerosene':
        f_hs = ( 0.8669 * _Ita + 0.091 ) / 0.796
        a_k,   b_k,   c_k   = 0.0005 * f_hs,  0.0028 * f_hs, 0.6818 * f_hs
        a_s,   b_s,   c_s   = 0.0024 * f_hs,  0.0021 * f_hs, 0.7560 * f_hs
        a_w,   b_w,   c_w   = 0.0005 * f_hs,  0.0028 * f_hs, 0.6818 * f_hs
        a_b1,  b_b1,  c_b1  = 0.0000 * f_hs, -0.0027 * f_hs, 0.9026 * f_hs
        a_b2,  b_b2,  c_b2  = 0.0000 * f_hs, -0.0024 * f_hs, 0.8885 * f_hs
        a_ba1, b_ba1, c_ba1 = 0.0000 * f_hs, -0.0027 * f_hs, 0.9026 * f_hs
        a_ba2, b_ba2, c_ba2 = 0.0062 * f_hs,  0.0462 * f_hs, 0.4001 * f_hs
        e_k   = min( a_k   * _theta_ex_ave + b_k   * ( _L_2dash_k + _L_2dash_w ) + c_k,   1.0 )
        e_s   = min( a_s   * _theta_ex_ave + b_s   *   _L_2dash_s                + c_s,   1.0 )
        e_w   = min( a_w   * _theta_ex_ave + b_w   * ( _L_2dash_k + _L_2dash_w ) + c_w,   1.0 )
        e_b1  = min( a_b1  * _theta_ex_ave + b_b1  *   _L_2dash_b1               + c_b1,  1.0 )
        e_b2  = min( a_b2  * _theta_ex_ave + b_b2  *   _L_2dash_b2               + c_b2,  1.0 )
        e_ba1 = min( a_ba1 * _theta_ex_ave + b_ba1 *   _L_2dash_ba1              + c_ba1, 1.0 )
        e_ba2 = min( a_ba2 * _theta_ex_ave + b_ba2 *   _L_2dash_ba2              + c_ba2, 1.0 )
    else:
        raise ValueError('バックアップボイラーの燃料種別に指定された値が不正です。')
    
    # 給湯（追い炊きありの場合）
    return  _L_2dash_k / e_k + _L_2dash_s / e_s + _L_2dash_w / e_w + _L_2dash_b2 / e_b2 + _L_2dash_ba2 / e_ba2

In [10]:
CGS_Spec('SOFC1')['BB_HW']['Ita']

0.905

#### Example

In [11]:
_theta = np.array([0.1, 2, 4])
_theta_ave = np.mean(_theta)

get_E_hs( _CGS_name='SOFC1', _theta_ex_ave = _theta_ave, _L_2dash_k=1, _L_2dash_s=2, _L_2dash_w=3, _L_2dash_b1=4, _L_2dash_b2=5, _L_2dash_ba1=6, _L_2dash_ba2=7 )

21.72929241309118

### 8.2 Hot Water Heat Load Shared by Backup Boiler / バックアップボイラーが分担する給湯熱負荷

$$
L_{BB,DHW,k} = L_{BB,DHW} \times \frac{L''_{k}}{L_{DHW}}
\tag{4a}
$$

$$
L_{BB,DHW,s} = L_{BB,DHW} \times \frac{L''_{s}}{L_{DHW}}
\tag{4b}
$$

$$
L_{BB,DHW,w} = L_{BB,DHW} \times \frac{L''_{w}}{L_{DHW}}
\tag{4c}
$$

$$
L_{BB,DHW,b1} = L_{BB,DHW} \times \frac{L''_{b1}}{L_{DHW}}
\tag{4d}
$$

$$
L_{BB,DHW,b2} = L_{BB,DHW} \times \frac{L''_{b2}}{L_{DHW}}
\tag{4e}
$$

$$
L_{BB,DHW,ba1} = L_{BB,DHW} \times \frac{L''_{ba1}}{L_{DHW}}
\tag{4f}
$$

$$
L_{BB,DHW,ba2} = L''_{ba2}
\tag{4g}
$$


Where  
$L_{BB,DHW}$ is the daily hot water supply heat load shared by backup boiler / 1日当たりの給湯のバックアップボイラーが分担する給湯熱負荷 ( MJ/d );  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷 ( MJ/d );  
$L''_{k}$ is the daily hot water supply hea load corrected for solar heat by the kitchen / 1日当たりの台所水栓における太陽熱補正給湯熱負荷 ( MJ/d );  
$L''_{w}$ is the daily hot water supply hea load corrected for solar heat by the washroom / 1日当たりの洗面水栓における太陽熱補正給湯熱負荷 ( MJ/d );  
$L''_{s}$ is the daily hot water supply hea load corrected for solar heat by the shower / 1日当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷 ( MJ/d );  
$L''_{b1}$ is the daily hot water supply hea load corrected for solar heat by filling the bathtub manually / 1日当たりの浴槽水栓湯はりにおける太陽熱補正給湯熱負荷 ( MJ/d );  
$L''_{b2}$ is the daily hot water supply hea load corrected for solar heat by filling the bathtub automattically / 1日当たりの浴槽自動湯はりにおける太陽熱補正給湯熱負荷 ( MJ/d );  
$L''_{ba1}$ is the daily hot water supply hea load corrected for solar heat by addition of hot water to bathtub / 1日当たりの浴槽水栓さし湯における太陽熱補正給湯熱負荷 ( MJ/d ).

In [12]:
# 式(4)
def get_L_BB( _L_BB_DHW, _L_DHW, _L_2dash_k, _L_2dash_s, _L_2dash_w, _L_2dash_b1, _L_2dash_b2, _L_2dash_ba1, _L_2dash_ba2):
    _L_BB_DHW_k   = _L_BB_DHW * _L_2dash_k   / _L_DHW
    _L_BB_DHW_s   = _L_BB_DHW * _L_2dash_s   / _L_DHW
    _L_BB_DHW_w   = _L_BB_DHW * _L_2dash_w   / _L_DHW
    _L_BB_DHW_b1  = _L_BB_DHW * _L_2dash_b1  / _L_DHW
    _L_BB_DHW_b2  = _L_BB_DHW * _L_2dash_b2  / _L_DHW
    _L_BB_DHW_ba1 = _L_BB_DHW * _L_2dash_ba1 / _L_DHW
    _L_BB_DHW_ba2 = _L_2dash_ba2
    
    return _L_BB_DHW_k, _L_BB_DHW_s, _L_BB_DHW_w, _L_BB_DHW_b1, _L_BB_DHW_b2, _L_BB_DHW_ba1, _L_BB_DHW_ba2


#### Exapmle

In [13]:
get_L_BB(10, 100, 1,2,3,4,5,6,0.1)

(0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.1)

$$\Large
L_{BB,DHW} = L_{DHW} - Q_{gen,DHW}
\tag{5}
$$

Where  
$Q_{gen,DHW}$ is the daily exhaust heat utilization of hot water supply / 1日当たりの給湯の排熱利用量 ( MJ/d ).  

In [14]:
# 式(5)
def get_L_BB_DHW( L_DHW, Q_gen_DHW ):
    return L_DHW - Q_gen_DHW

#### Example

In [15]:
get_L_BB_DHW(10,4)

6

## 9.Fuel Consumption of Backup Boiler Hot Water Heating Supply / 温水暖房時のバックアップボイラーの燃料消費量

$$\Large
E_{F,BB,HWH} = \sum_t E_{F,BB,HWH,t}
\tag{6}
$$

Where  
$E_{F,BB,HWH}$ is the daily fuel consumption of backup boiler during hot water heating ( MJ/d )  
$E_{F,BB,HWH,t}$ is the hourly fuel consumption of backup boiler during hot water heating at $t$ o'clock / 時刻$t$における1時間当たりの温水暖房時のバックアップボイラーの燃料消費量 ( MJ/h ).  

In [16]:
# 式(6)
def get_E_F_BB_HWH(_CGS_name, _Q_out_BB_HWH_t):
    
    _Type = CGS_Spec(_CGS_name)['BB_WH']['Type']
    _Ita =  CGS_Spec(_CGS_name)['BB_WH']['Ita']
    _P_rtd =  CGS_Spec(_CGS_name)['BB_WH']['RatedPower']
    
    _theta = 40 # 暫定の送水温度
    
    if _Type == 'kerosene': # 付録A　石油熱源機
        _theta_RW_hs = 40 # 暫定
        _Q_body = _04_07_AppA_ker.get_Q_body(_equipment = _Type, _theta_SW_hs = _theta)
        _f_rtd = _04_07_AppA_ker.get_f_rtd(_equipment = _Type, _theta_RW_hs = _theta_RW_hs)
        _Q_body_rtd = _04_07_AppA_ker.get_Q_body_rtd(_equipment = _Type)
        _e_ex = _04_07_AppA_ker.get_e_ex(_equipment = _Type, _e_rtd = _Ita, _f_rtd = _f_rtd, _q_rtd_hs = _P_rtd, _Q_body_rtd = _Q_body_rtd, _theta_SW_hs = _theta)
        _E_F_BB_HWH_t = _04_07_AppA_ker.get_E_K_hs(_Q_out_H_hs = _Q_out_BB_HWH_t, _Q_body = _Q_body, _e_ex = _e_ex)
        
    elif _Type == 'gasCondensing' or _Type == 'GasConv': # 付録B ガス熱源機
        _Q_body = _04_07_AppB_gas.get_Q_body(_equipment = _Type, _supply_theta = _theta)
        _e_ex = _04_07_AppB_gas.get_e_ex(_equipment = _Type, _e_rtd = _Ita, _q_rtd_hs = _P_rtd, _Q_body = _Q_body, _supply_theta = _theta)
        _E_F_BB_HWH_t = _04_07_AppB_gas.get_E_G_hs(_Q_out_H_hs = _Q_out_BB_HWH_t, _Q_body = _Q_body, _e_ex = _e_ex)
    
    
    return np.sum(_E_F_BB_HWH_t) # 24時間分を合計

#### Example

In [17]:
get_E_F_BB_HWH(_CGS_name = 'SOFC2', _Q_out_BB_HWH_t = 10)

11.20441802210468

In [18]:
# 式(7)
def get_Q_out_BB_HWH_t(_CGS_name, _L_BB_HWH_t):

    _Type = CGS_Spec(_CGS_name)['BB_WH']['Type']
    _P_rtd = CGS_Spec(_CGS_name)['BB_WH']['RatedPower']
    
    if _Type == 'kerosene': # 付録A　石油熱源機
        _Q_max_BB_HWH = _04_07_AppA_ker.get_Q_max_H_hs(_q_rtd_hs = _P_rtd)
    elif _Type == 'gasCondensing' or 'GasConv': # 付録B ガス熱源機
        _Q_max_BB_HWH = _04_07_AppB_gas.get_Q_max_H_hs(_q_rtd_hs = _P_rtd)

    return np.minimum(_L_BB_HWH_t, _Q_max_BB_HWH)

    

#### Example

In [19]:
get_Q_out_BB_HWH_t('PEFC5', 1000)

62.64

$$\large
L_{BB,HWH,t} = \left\{ \begin{array}{ll}
    L_{HWH,t} - Q_{gen,HWH} \times L_{HWH,t} \div L_{HWH} & (L_{HWH} \neq 0)\\ 
    0 & (L_{HWH} = 0)\\ 
    \end{array}
\right.
\tag{8}
$$

Where  
$L_{BB,HWH,t}$ is the hourly heat load of hot water heating shared by backup boiler during hot water heating. / 1時間当たりの温水暖房時のバックアップボイラーが分担する温水暖房の熱負荷(MJ/day)    
$L_{HWH,t}$ is the hourly heat load of hot water heating at $t$ o'clock. / 時刻$t$における1時間当たりの温水暖房の熱負荷(MJ/h)    
$Q_{gen,HWH}$ is the daily exhaust heat consumption of hot water heating. / 1日当たりの温水暖房の排熱利用量(MJ/day)   
$L_{HWH}$ is the daily heat load of hot water heating. / 1日当たりの温水暖房の熱負荷(MJ/day)    

In [20]:
# 式(8)
def get_L_BB_HWH_t(_L_HWH_t, _Q_gen_HWH, _L_HWH):
    if _L_HWH == 0:
        return 0
    elif _L_HWH != 0:
        return np.array(_L_HWH_t) - _Q_gen_HWH * np.array(_L_HWH_t) / _L_HWH
    else:
        raise ValueError('_L_HWH に不正な値が指定されました。')

#### Example

In [21]:
get_L_BB_HWH_t(L_HWH_t = (10,20), _Q_gen_HWH = 5, _L_HWH = 100)

array([ 9.5, 19. ])

## 10.Power Generation Unit / 発電ユニット

### 10.1 Exhaust Heat Utilization / 排熱利用量

The daily exhaust heat utilization $Q_{gen,DHW}$ of hot water supply and the daily exhaust heat utilization $Q_{gen,HWH}$ of hot water heating are calculated by the equations below, depending on whether exhaust heat is used for hot water heating and how exhaust heat is used.   
Whether exhaust heat is used for hot water heating and how exhaust heat is used are in accordance with Appendix A.

&#9312;When there is no use of exhaust heat to hot water heating:<br><br>
$$\large
Q_{gen,DHW} = min(Q_{PU,gen} \times r_{DHW,gen,PU},L_{DHW})
\tag{9a}
$$

$$\large
Q_{gen,HWH} = 0.0
\tag{9b}
$$
<br>

&#9313;In case of using exhaust heat to hot water heating, models with hot water supply priority: <br><br>
$$\large
Q_{gen,DHW} = min(Q_{PU,gen} \times r_{DHW,gen,PU},L_{DHW})
\tag{10a}
$$

$$\large
Q_{gen,HWH} = min((Q_{PU,gen} - Q_{gen,DHW}) \times r_{HWH,gen,PU},L_{HWH})
\tag{10b}
$$
<br>

&#9314;In case of using exhaust heat to hot water heating, models with hot water heating priority: <br><br>
$$\large
Q_{gen,HWH} = min(Q_{PU,gen} \times r_{HWH,gen,PU},L_{HWH})
\tag{11a}
$$

$$\large
Q_{gen,DHW} = min((Q_{PU,gen} - Q_{gen,HWH}) \times r_{DHW,gen,PU},L_{DHW})
\tag{11b}
$$
<br>


Where  
$Q_{PU,gen}$ is the daily exhaust heat quantity of power generation unit. / 1日当たりの発電ユニット排熱量(MJ/day)  
$r_{DHW,gen,PU}$ is the exhaust heat utilization rate at hot water supply in power generation unit. / 発電ユニットの給湯排熱利用率  
$r_{HWH,gen,PU}$ is the exhaust heat utilization rate at hot water heating in power generation unit. / 発電ユニットの温水暖房排熱利用率  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub. / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/day)  

The exhaust heat utilization rate $r_{DHW,gen,PU}$ at hot water supply and the exhaust heat utilization rate $r_{HWH,gen,PU}$ at hot water heating in power generation unit are in accordance with Appendix A.

In [22]:
# 式(9)-(11)
def get_Q_gen_DHWandHWH(_CGS_name, _Q_PU_gen, _L_DHW, _L_HWH):
    exaust_use_WH = CGS_Spec(_CGS_name)['ExaustUseForWaterHeating']
    exaust_priority = CGS_Spec(_CGS_name)['ExaustUsePriority']
    r_DHW_gen_PU = CGS_Spec(_CGS_name)['PU_exaust_rate']['HotWater']
    r_HWH_gen_PU = CGS_Spec(_CGS_name)['PU_exaust_rate']['WaterHeating']
    
    if exaust_use_WH == 'No':
        _Q_gen_DHW = min(_Q_PU_gen * r_DHW_gen_PU, _L_DHW)
        _Q_gen_HWH = 0
    elif exaust_use_WH == 'Yes' and exaust_priority == 'HotWater':
        _Q_gen_DHW = min(_Q_PU_gen * r_DHW_gen_PU, _L_DHW)
        _Q_gen_HWH = min((_Q_PU_gen - _Q_gen_DHW) * r_HWH_gen_PU, _L_HWH)
    elif exaust_use_WH == 'Yes' and exaust_priority == 'WaterHeating':
        _Q_gen_HWH = min(_Q_PU_gen * r_HWH_gen_PU, _L_HWH)
        _Q_gen_DHW = min((_Q_PU_gen - _Q_gen_HWH) * r_DHW_gen_PU, _L_DHW)
    return _Q_gen_DHW, _Q_gen_HWH

#### Example

In [23]:
print(get_Q_gen_DHWandHWH(_CGS_name = 'GEC1' , _Q_PU_gen = 20, _L_DHW = 3, _L_HWH = 13))
print(get_Q_gen_DHWandHWH(_CGS_name = 'PEFC5', _Q_PU_gen = 20, _L_DHW = 3, _L_HWH = 13))
print(get_Q_gen_DHWandHWH(_CGS_name = 'SOFC2', _Q_PU_gen = 20, _L_DHW = 3, _L_HWH = 13))

(3, 13)
(3, 0.0)
(3, 0)


The daily exhaust heat quantity $Q_{PU,gen}$ of power generation unit is calculated by the equation below.

$$\large
Q_{PU,gen} = E_{F,PU} \times e_{H,PU}
\tag{12}
$$

Where  
$E_{F,PU}$ is the daily fuel consumption of power generation unit. / 1日当たりの発電ユニットの燃料消費量(MJ/day)  
$e_{H,PU}$ is the daily average exhaust heat efficiency of power generation unit. / 発電ユニットの日平均排熱効率  

In [24]:
# 式(12)
get_Q_PU_gen = lambda _E_F_PU, _e_H_PU : _E_F_PU * _e_H_PU
    

### 10.2 Amount of Generation / 発電量

The hourly power generation $E_{E,gen,PU,t}$ of power generation unit at $t$ o'clock is calculated by the equation below.

$$\large
E_{E,gen,PU,t} = E_{E,gen,PU} \times \frac{E_{E,PU,t}}{E_{E,PU}}
\tag{13}
$$

Where  
$E_{E,gen,PU}$ is the daily power generation of power generation unit. / 1日当たりの発電ユニットの発電量(kWh/day)  
$E_{E,PU,t}$ is the hourly sharing power load of power generation unit. / 1時間当たりの発電ユニットの分担可能電力負荷(kWh/h)  
$E_{E,PU}$ is the daily power load that can be shared by power generation unit. / 1日当たりの発電ユニットの分担可能電力負荷(kWh/day)  

In [25]:
# 式(13)
def get_E_E_gen_PU_t(_E_E_gen_PU, _E_E_PU_t, _E_E_PU):
    return _E_E_gen_PU * np.array(_E_E_PU_t) / _E_E_PU
    

#### Example

The power generation $E_{E,gen,PU}$ of power generation unit is calculated by the equation below.

$$\large
E_{E,gen,PU} = E_{F,PU} \times e_{E,PU} \div 3.6
\tag{14}
$$

Where  
$E_{F,PU}$ is the daily fuel consumption of power generation unit. / 1日当たりの発電ユニットの燃料消費量(MJ/day)  
$e_{E,PU}$ is the daily average power generation efficiency of power generation unit. / 発電ユニットの日平均発電効率  

In [26]:
# 式(14)
def get_E_E_gen_PU(_E_F_PU, _e_E_PU):
    return _E_F_PU * _e_E_PU / 3.6


#### Example

In [27]:
get_E_E_gen_PU(_E_F_PU = 360, _e_E_PU = 0.2)

20.0

### 10.3 Fuel Consumption / 燃料消費量

The daily fuel consumption $E_{F,PU}$ of power generation unit is calculated by the equations below.  
The power generation method of the power generation unit is accordance with Appendix A.

&#9312; When the power generation method of the power generation unit is "heat main":  

$$\large
E_{F,PU} = min(E_{F,PU,EVt}, E_{F,PU,HVt})
\tag{15a}
$$

&#9313; When the power generation method of the power generation unit is "electricity main":  

$$\large
E_{F,PU} = E_{F,PU,EVt}
\tag{15b}
$$

Where  
$E_{F,PU,EVt}$ is the virtual daily fuel consumption at estimation of power generation amount by power generation unit. / 1日当たりの発電ユニットの発電量推定時の仮想燃料消費量(MJ/day)  
$E_{F,PU,HVt}$ is the virtual daily fuel consumption at estimation of exhaust heat quantity by power generation unit. / 1日当たりの発電ユニットの排熱量推定時の仮想燃料消費量(MJ/day)  

In [28]:
# 式(15)
def get_E_F_PU(_CGS_name, _E_F_PU_EVt, _E_F_PU_HVt):
    
    generate_use = CGS_Spec(_CGS_name)['PU_GenerationMethod']
    
    if generate_use == 'HeatPriority':
        return min(_E_F_PU_EVt, _E_F_PU_HVt)
    elif generate_use == 'ElectricityPriority':
        return _E_F_PU_EVt
    else:
        raise ValueError('_generate_use に不正な値が指定されました。')

#### Example

In [29]:
print("get_E_F_PU:", get_E_F_PU(_CGS_name = 'GEC1', _E_F_PU_EVt = 30, _E_F_PU_HVt = 20))
print("get_E_F_PU:", get_E_F_PU(_CGS_name = 'SOFC2', _E_F_PU_EVt = 30, _E_F_PU_HVt = 20))

get_E_F_PU: 20
get_E_F_PU: 30


The virtual daily fuel consumption $E_{F,PU,EVt}$ at estimation of power generation amount by power generation unit is calculated by the equation below.

$$\large
E_{F,PU,EVt} = E_{E,gen,PU,EVt} \times 3.6 \div e_{E,PU} 
\tag{16}
$$

Where  
$E_{E,gen,PU,EVt}$ is the virtual daily generation amount at estimation of power generation amount by power generation unit. / 1日当たりの発電ユニットの発電量推定時の仮想発電量(kWh/day)  
$e_{E,PU}$ is the daily average power generation efficiency of power generation unit. / 発電ユニットの日平均発電効率  

In [30]:
# 式(16)
def get_E_F_PU_EVt(_E_E_gen_PU_EVt, _e_E_PU):
    return _E_E_gen_PU_EVt * 3.6 / _e_E_PU

The virtual daily generation amount $E_{E,gen,PU,EVt}$ at estimation of the power generation amount of the power generation unit is calculated by the equation below.

$$\large
E_{E,gen,PU,EVt} = min(a_{PU} \times E_{E,PU} \times 3.6 + a_{DHW} \times L_{DHW} + a_{HWH} \times L_{HWH}+ b,\\
\large
E_{E,PU} \times c \times 3.6) \div 3.6
\tag{17}
$$

Where  
$E_{E,PU}$ is the daily power load that can be shared by power generation unit. / 1日当たりの発電ユニットの分担可能電力負荷(kWh/day)  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub. / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/day)  
$L_{HWH}$ is the daily heat load of hot water heating. / 1日当たりの温水暖房の熱負荷(MJ/day)  

In [31]:
# 式(17)
def get_E_E_gen_PU_EVt(_CGS_name, _E_E_PU, _L_DHW, _L_HWH):
    a_PU = CGS_Spec(_CGS_name)['PU_virtualGeneration']['a_PU']
    a_DHW = CGS_Spec(_CGS_name)['PU_virtualGeneration']['a_DHW']
    a_HWH = CGS_Spec(_CGS_name)['PU_virtualGeneration']['a_HWH']
    b =  CGS_Spec(_CGS_name)['PU_virtualGeneration']['b']
    c = CGS_Spec(_CGS_name)['PU_virtualGeneration']['c']
    
    a = a_PU * _E_E_PU * 3.6 + a_DHW * _L_DHW + a_HWH * _L_HWH + b
    b = _E_E_PU * c * 3.6
    return min(a, b) / 3.6

#### Example

In [32]:
get_E_E_gen_PU_EVt(_CGS_name = 'PEFC5', _E_E_PU = 10, _L_DHW = 5, _L_HWH = 3)

8.324194444444442

The coefficients $a_{PU}$, $a_{DHW}$, $a_{HWH}$, $b$ and $c$ are accordance with Appendix A.

The virtual daily fuel consumption $E_{F,PU,HVt}$ at estimation of exhaust heat quantity by power generation unit is calculated by the equation below only for models using the equation (17a).

$$\large
E_{F,PU,HVt} = (a_{DHW} \times L_{DHW} + a_{HWH} \times L_{HWH}) \times r_{H,gen,PU,HVt} \div e_{H,PU}
\tag{18}
$$

Where  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub. / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/day)  
$L_{HWH}$ is the daily heat load of hot water heating. / 1日当たりの温水暖房の熱負荷(MJ/day)  
$r_{H,gen,PU,HVt}$ is the virtual exhaust heat quantity upper limit ratio at estimation of exhaust heat quantity by power generation unit. / 発電ユニットの排熱量推定時の仮想排熱量上限比  
$e_{H,PU}$ is the daily average exhaust heat efficiency of power generation unit. / 発電ユニットの日平均排熱効率  

The coefficients $a_{DHW}$ and $a_{HWH}$ are accordance with Appendix A.

In [33]:
# 式(18)
def get_E_F_PU_HVt(_CGS_name, _L_DHW, _L_HWH, _r_H_gen_PU_HVt, _e_H_PU):
    a_DHW = CGS_Spec(_CGS_name)['PU_virtualFuel']['a_DHW']
    a_HWH = CGS_Spec(_CGS_name)['PU_virtualFuel']['a_HWH']
    
    return (a_DHW * _L_DHW + a_HWH * _L_HWH) + _r_H_gen_PU_HVt / _e_H_PU
    

In [34]:
get_E_F_PU_HVt(_CGS_name = 'PEFC5', _L_DHW = 10, _L_HWH = 20, _r_H_gen_PU_HVt = 0.3, _e_H_PU = 0.8)

10.375

The virtual exhaust heat quantity upper limit ratio $r_{H,gen,PU,HVt}$ at estimation of exhaust heat quantity by power generation unit is calculated by the equation below.

$$\large
r_{H,gen,PU,HVt} = a_{DHW} \times L_{DHW} + a_{HWH} \times L_{HWH} + b
\tag{19}
$$

Where  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub. / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/day)  
$L_{HWH}$ is the daily heat load of hot water heating. / 1日当たりの温水暖房の熱負荷(MJ/day)  

The coefficients $a_{DHW}$, $a_{HWH}$ and $b$ are accordance with Appendix A.

In [35]:
# 式(19)
def get_r_H_gen_PU_HVt(_CGS_name, _L_DHW, _L_HWH):
    a_DHW = CGS_Spec(_CGS_name)['PU_UpperCoef']['a_DHW']
    a_HWH = CGS_Spec(_CGS_name)['PU_UpperCoef']['a_HWH']
    b     = CGS_Spec(_CGS_name)['PU_UpperCoef']['b']
    
    return a_DHW * _L_DHW * a_HWH * _L_HWH + b


In [36]:
get_r_H_gen_PU_HVt(_CGS_name = 'PEFC5', _L_DHW = 10, _L_HWH = 20)

1.0526

### 4. Power Generation Efficiency / 発電効率

The daily average power generation efficiency $e_{E,PU}$ of power generation unit is calculated by the equation below.  
However, the value obtained by the following equation is assumed to be equal to the upper limit value when it exceeds the upper limit value of the power generation efficiency of the power generation unit and equal to the lower limit value if it is lower than the lower limit value.

$$\large
e_{E,PU} = a_{PU} \times E_{E,PU} \times 3.6 + a_{DHW} \times L_{DHW} + a_{HWH} \times L_{HWH} + b
\tag{20}
$$

Where  
$E_{E,PU}$ is the daily power load that can be shared by power generation unit. / 1日当たりの発電ユニットの分担可能電力負荷(kWh/day)  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub. / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/day)  
$L_{HWH}$ is the daily heat load of hot water heating. / 1日当たりの温水暖房の熱負荷(MJ/day)  

The coefficients $a_{PU}$, $a_{DHW}$, $a_{HWH}$, $b$, the upper limit value and the lower limit value are accordance with Appendix A.

In [37]:
# 式(20)
def get_e_E_PU(_CGS_name, _E_E_PU, _L_DHW, _L_HWH):
    a_PU = CGS_Spec(_CGS_name)['PU_GenIta']['a_PU']
    a_DHW = CGS_Spec(_CGS_name)['PU_GenIta']['a_DHW']
    a_HWH = CGS_Spec(_CGS_name)['PU_GenIta']['a_HWH']
    b = CGS_Spec(_CGS_name)['PU_GenIta']['b']
    upperValue =CGS_Spec(_CGS_name)['PU_GenIta']['UpperLimit']
    lowerValue = CGS_Spec(_CGS_name)['PU_GenIta']['LowerLimit']
    
    e_E_PU = a_PU * _E_E_PU * 3.6 + a_DHW * _L_DHW + a_HWH * _L_HWH + b
    
    return max(min(e_E_PU, upperValue), lowerValue)
    

#### Example

In [38]:
get_e_E_PU(_CGS_name = 'PEFC5', _E_E_PU = 10, _L_DHW = 30, _L_HWH = 50)

0.259756

### 10.5 Exhaust Heat Efficiency / 排熱効率

The daily average exhaust heat efficiency $e_{H,PU}$ of power generation unit is calculated by the equation below.  
However, the value obtained by the following equation is assumed to be equal to the upper limit value when it exceeds the upper limit value of the exhaust heat efficiency of the power generation unit and equal to the lower limit value if it is lower than the lower limit value.

$$\large
e_{H,PU} = a_{PU} \times E_{E,PU} \times 3.6 + a_{DHW} \times L_{DHW} + a_{HWH} \times L_{HWH} + b
\tag{21}
$$

Where  
$E_{E,PU}$ is the daily power load that can be shared by power generation unit. / 1日当たりの発電ユニットの分担可能電力負荷(kWh/day)  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub. / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/day)  
$L_{HWH}$ is the daily heat load of hot water heating. / 1日当たりの温水暖房の熱負荷(MJ/day)  

The coefficients $a_{PU}$, $a_{DHW}$, $a_{HWH}$, $b$, the upper limit value and the lower limit value are accordance with Appendix A.

In [39]:
# 式(21)
def get_e_H_PU(_CGS_name, _E_E_PU, _L_DHW, _L_HWH):
    a_PU = CGS_Spec(_CGS_name)['PU_ExaustIta']['a_PU']
    a_DHW = CGS_Spec(_CGS_name)['PU_ExaustIta']['a_DHW']
    a_HWH = CGS_Spec(_CGS_name)['PU_ExaustIta']['a_HWH']
    b = CGS_Spec(_CGS_name)['PU_ExaustIta']['b']
    upperValue = CGS_Spec(_CGS_name)['PU_ExaustIta']['UpperLimit']
    lowerValue = CGS_Spec(_CGS_name)['PU_ExaustIta']['LowerLimit']
    
    e_H_PU = a_PU * _E_E_PU * 3.6 + a_DHW * _L_DHW + a_HWH * _L_HWH + b
    
    return max(min(e_H_PU, upperValue), lowerValue)
    

In [40]:
get_e_H_PU(_CGS_name = 'PEFC5', _E_E_PU = 20, _L_DHW = 3, _L_HWH = 4)

0.30305600000000005

### 10.6 Shareable Electric Load / 分担可能電力負荷

The daily power load $E_{E,PU}$ that can be shared by power generation unit is calculated by the equation below.

$$\large
E_{E,PU} = \sum E_{E,PU,t}
\tag{22}
$$

Where  
$E_{E,PU,t}$ is the hourly power load that can be shared by power generation unit at $t$ o'clock. / 時刻$t$における1時間当たりの発電ユニットの分担可能電力負荷(kWh/h)  

In [41]:
# 式(22)
def get_E_E_PU(_E_E_PU_t):
    return np.sum(_E_E_PU_t)

#### Example

In [42]:
a =(1)
get_E_E_PU(a)

1

The hourly power load $E_{E,PU,t}$ that can be shared by power generation unit at $t$ o'clock is calculated by the equation below.

$$\large
E_{E,PU,t} = min(E_{E,dmd,PU,t}, P_{rtd,PU} \times 10^{-3})
\tag{23}
$$

Where  
$E_{E,dmd,PU,t}$ is the hourly power demand of power generation unit at $t$ o'clock. / 時刻$t$における1時間当たりの発電ユニットの電力需要(kWh/h)  
$P_{rtd,PU}$ is the rated power generation output. / 定格発電出力(W)  

The rated power generation output $P_{rtd,PU}$ is accordance with Appendix A.

In [43]:
# 式(23)
def get_E_E_PU_t(_CGS_name, _E_E_dmd_PU_t):
    P_rtd_PU = CGS_Spec(_CGS_name)['RatedPower']
    
    return np.minimum(np.array(_E_E_dmd_PU_t), P_rtd_PU * 10**(-3))

#### Example

In [44]:
print(get_E_E_PU_t(_CGS_name = 'GEC1', _E_E_dmd_PU_t = np.array([0.3,0.4])))
print(get_E_E_PU_t(_CGS_name = 'PEFC5', _E_E_dmd_PU_t = [1.4,5]))

[0.3 0.4]
[0.7 0.7]


The power demand $E_{E,dmd,PU}$ of power generation unit is calculated by the equations below.

$$\large
E_{E,dmd,PU,t} = E_{E,dmd,t} + E_{E,TU,aux,t}
\tag{24}
$$


Where  
$E_{E,dmd,t}$ is the hourly power demand. / 1時間当たりの電力需要(kWh/h)  
$E_{E,TU,aux,t}$ is the hourly auxiliary machine power consumption of the tank unit. / 1時間当たりのタンクユニットの補機消費電力量(kWh/h)  

In [45]:
# 式(24)
def get_E_E_dmd_PU_t(_E_E_dmd_t, _E_E_TU_aux_t):
    return _E_E_dmd_t + _E_E_TU_aux_t

## 12.Power Consumption of Auxiliaries of Tank Unit / タンクユニットの補機消費電力

The hourly power consumption $E_{E,TU,aux}$ of auxiliaries of tank unit is calculated by the equation below.

$$\large
E_{E,TU,aux,t} = E_{E,TU,aux,DHW,t} + E_{E,TU,aux,HWH,t} + E_{E,TU,aux,ba2,t}
\tag{25}
$$

$$\large
E_{E,TU,aux,DHW,t} = P_{TU,aux,DHW} \times 10^{-3}
\tag{26}
$$

①温水暖房への排熱利用がある場合
$$
E_{E,TU,aux,HWH,t} = P_{TU,aux,HWH} \times r_{WS,HWH,t} \times 10^{-3}
\tag{27}
$$

In [46]:
# 式(25)
def get_E_E_TU_aux_t(_E_E_TU_aux_DHW_t, _E_E_TU_aux_HWH_t, _E_E_TU_aux_ba2_t):
    return _E_E_TU_aux_DHW_t + _E_E_TU_aux_HWH_t + _E_E_TU_aux_ba2_t

In [47]:
# 式(26)
def get_E_E_TU_aux_DHW_t(_CGS_name):
    return CGS_Spec(_CGS_name)['TU_auxPower']['HotWater'] * 10**(-3)
    

In [48]:
# 式(27)
def get_E_E_TU_aux_HWH_t(_CGS_name, _L_HWH_t, _r_WS_ln_i):
    
    exaust_use_WH = CGS_Spec(_CGS_name)['ExaustUseForWaterHeating']
    P_TU_aux_HWH = CGS_Spec(_CGS_name)['TU_auxPower']['WaterHeating']
    _equipment = CGS_Spec(_CGS_name)['BB_WH']['Type']
    _e_rtd = CGS_Spec(_CGS_name)['BB_WH']['Ita']
    _q_rtd_hs = CGS_Spec(_CGS_name)['BB_WH']['RatedPower']
    theta = 60 # 暫定
    
    # 温水供給運転率
    _r_WS_HWH_t = _04_07_MainPart.get_r_WS_hs(_Q_dmd_H_hs = _L_HWH_t, _r_WS_ln_i = _r_WS_ln_i)
    
    if exaust_use_WH == 'Yes':
        return P_TU_aux_HWH * _r_WS_HWH_t * 10**(-3)
    elif exaust_use_WH == 'No':
        _Q_gen_HWH = 0 #温水暖房への排熱利用がないため0となる。
        _L_BB_HWH_t = _L_HWH_t - _Q_gen_HWH
        _Q_out_BB_HWH_t = get_Q_out_BB_HWH_t(_CGS_name, _L_BB_HWH_t)
        
        # 付録A　石油熱源機の場合は未作成
        
        # 付録B　ガス熱源機の場合
        _Q_body = _04_07_AppB_gas.get_Q_body(_equipment , _supply_theta = theta) 
        _e_ex = _04_07_AppB_gas.get_e_ex(_equipment, _e_rtd, _q_rtd_hs, _Q_body, _supply_theta = theta)
        _E_G_hs = _04_07_AppB_gas.get_E_G_hs(_Q_out_H_hs = _Q_out_BB_HWH_t, _Q_body = _Q_body, _e_ex = _e_ex)

        _E_E_hs_pmp = _04_07_AppB_gas.get_E_E_hs_pmp(_r_WS_hs = _r_WS_HWH_t)
        _E_E_hs_fan = _04_07_AppB_gas.get_E_E_hs_fan(_E_G_hs)
        _E_E_TU_aux_HWH_t = _04_07_AppB_gas.get_E_E_hs(_E_E_hs_pmp, _E_E_hs_fan)
        return _E_E_TU_aux_HWH_t
        
    else:
        raise ValueError('不正な値が入力されています。')


In [49]:
CGS_Spec("SOFC2")

{'BB_HW': {'Ita': 0.905, 'Type': 'gas'},
 'BB_WH': {'Ita': 0.87, 'RatedPower': 17400, 'Type': 'gasCondensing'},
 'ExaustUseForWaterHeating': 'No',
 'ExaustUsePriority': 'N/A',
 'PU_ExaustIta': {'LowerLimit': 0.2756,
  'UpperLimit': 0.3179,
  'a_DHW': 0.0,
  'a_HWH': 0.0,
  'a_PU': 0.0028,
  'b': 0.1854},
 'PU_GenIta': {'LowerLimit': 0.3503,
  'UpperLimit': 0.429,
  'a_DHW': 0.0,
  'a_HWH': 0.0,
  'a_PU': 0.0058,
  'b': 0.1611},
 'PU_GenerationMethod': 'ElectricityPriority',
 'PU_UpperCoef': {'a_DHW': 0, 'a_HWH': 0, 'b': 0},
 'PU_exaust_rate': {'HotWater': 0.6885, 'WaterHeating': 0.0},
 'PU_virtualFuel': {'a_DHW': 0, 'a_HWH': 0},
 'PU_virtualGeneration': {'a_DHW': 0.0,
  'a_HWH': 0.0,
  'a_PU': 1.1262,
  'b': -6.5572,
  'c': 0.9846},
 'RatedPower': 700,
 'TU_auxPower': {'HotWater': 11.8, 'WaterHeating': 0}}

In [50]:
get_E_E_TU_aux_HWH_t(_CGS_name = 'SOFC2', _L_HWH_t = 10, _r_WS_ln_i = [0.7, None, None, None, None])

0.0511000098487057

In [51]:
# 式(28)
def get_E_E_TU_aux_ba2_t(_CGS_name, time, _L_2dash_ba2):
    
    _BB_WH_Type = CGS_Spec(_CGS_name)['BB_WH']['Type']
    
    if time == 19:
        if _BB_WH_Type == 'gasCondensing' or _BB_WH_Type == 'gasConv':
            return _07_AppC_gas.get_E_E_hs_aux3(_L_2dash_ba2)
        elif _BB_WH_Type == 'kerCondensing' or _BB_WH_Type == 'gasConv':
            return _07_AppD_ker.get_E_E_hs_aux3(_L_2dash_ba2)
    elif time != 19:
        return 0
    

#### Exapmle

In [52]:
get_E_E_TU_aux_ba2_t(_CGS_name='SOFC1', time = 19, _L_2dash_ba2 = 10)

0.06480277777777778

## Others / その他

The daily hot water supply heat load $L_{DHW}$ except rewarming the bathtub is calculated by the equation below.

$$\large
L_{DHW} = L''_{k} + L''_{w} + L''_{s} + L''_{b1} + L''_{b2} + L''_{ba1}
\tag{29}
$$

Where  
$ L''_{k} $ is the daily hot water supply heat load corrected for solar heat by the kitchen./1日当たりの台所水栓の太陽熱補正給湯熱負荷(MJ/day)  
$ L''_{w} $ is the daily hot water supply heat load corrected for solar heat by the washroom./1日当たりの洗面水栓の太陽熱補正給湯熱負荷(MJ/day)  
$ L''_{s} $ is the daily hot water supply heat load corrected for solar heat by the shower./1日当たりの浴室シャワー水栓の太陽熱補正給湯熱負荷(MJ/day)  
$ L''_{b1} $ is the daily hot water supply heat load corrected for solar heat by filling the bathtub manually./1日当たりの浴槽水栓湯はりの太陽熱補正給湯熱負荷(MJ/day)  
$ L''_{b2} $ is the daily hot water supply heat load corrected for solar heat by filling the bathtub automattically./1日当たりの浴槽自動湯はりの太陽熱補正給湯熱負荷(MJ/day)  
$ L''_{ba1} $ is the daily hot water supply heat load corrected for solar heat by addition of hot water to bathtub./1日当たりの浴槽水栓さし湯の太陽熱補正給湯熱負荷(MJ/day)  

The daily hot water supply heat load $L''_{u}$ corrected for solar heat (where subscript $u$ is a suffix for use ($k$, $s$, $w$, $b1$, $b2$, $ba1$, $ba2$)) shall be the value calculated by Chapter 7 according to the number of residents.  
The number of residents mentioned here is not the actual number of people living in the dwelling unit but the number of resident virtually determined from the total of the floor area of the dwelling unit, and be obtained from Appendix C in Chapter 2.

In [53]:
# 式(29)
def get_L_DHW(_L_2dash_k, _L_2dash_w, _L_2dash_s, _L_2dash_b1, _L_2dash_b2, _L_2dash_ba1):
    return _L_2dash_k + _L_2dash_w + _L_2dash_s + _L_2dash_b1 + _L_2dash_b2 + _L_2dash_ba1

The daily heat load $L_{HWH}$ of hot water heating is calculated by the equation below.

$$
L_{HWH} = \sum^{23}_{t=0}L_{HWH,t}
\tag{30}
$$

Where  
$L_{HWH,t}$ is the heat load of hot water heating per hour at $t$ o'clock. / ݇時刻$t$における1 時間当たりの温水暖房の熱負荷(MJ/day)

In [54]:
# 式(30)
def get_L_HWH(_L_HWH_t):
    return np.sum(_L_HWH_t)

In [55]:
a = range(1,15)
get_L_HWH(a)

105

In [56]:
# 式(31)
def get_E_E_dmd_t(_E_E_H_t, _E_E_C_t, _E_E_V_t, _E_E_L_t, _E_E_AP_t):
    return _E_E_H_t + _E_E_C_t + _E_E_V_t + _E_E_L_t + _E_E_AP_t

The hourly power demand $E_{E,dmd}$ is calculated by the equation below.

$$\large
E_{E,dmd} = E_{E,H} + E_{E,C} + E_{E,V} + E_{E,L} + E_{E,AP}
\tag{31}
$$

Where  
$E_{E,dmd}$ is the hourly power demand. / 1時間当たりの電力需要(kWh/h)  
$E_{E,H}$ is the hourly power consumption of heating equipment. / 1時間当たりの暖房設備の消費電力量(kWh/h)  
$E_{E,C}$ is the hourly power consumption of cooling equipment. / 1時間当たりの冷房設備の消費電力量(kWh/h)  
$E_{E,V}$ is the hourly power consumption of mechanical ventilation equipment. / 1時間当たりの機械換気設備の消費電力量(kWh/h)  
$E_{E,L}$ is the hourly power consumption of lighting equipment. / 1時間当たりの照明設備の消費電力量(kWh/h)  
$E_{E,AP}$ is the hourly power consumption of home appliances. / 1時間当たりの家電の消費電力量(kWh/h)  

## Integrated Function

In [66]:
def get_result(time, _CGS_name, _theta_ex, _E_E_H_t, _E_E_C_t, _E_E_V_t, _E_E_L_t, _E_E_AP_t, _L_HWH_t, _L_2dash_k, _L_2dash_w, _L_2dash_s, _L_2dash_b1, _L_2dash_b2, _L_2dash_ba1, _L_2dash_ba2, _r_WS_ln_i):
    
    # その他
    _E_E_dmd_t = get_E_E_dmd_t(_E_E_H_t, _E_E_C_t, _E_E_V_t, _E_E_L_t, _E_E_AP_t)
    _L_HWH = get_L_HWH(_L_HWH_t)
    _L_DHW = get_L_DHW(_L_2dash_k, _L_2dash_w, _L_2dash_s, _L_2dash_b1, _L_2dash_b2, _L_2dash_ba1)
    # _L_BB_ba2 = get_L_BB_ba2(_L_2dash_ba2)
    
    # print({'_E_E_dmd':_E_E_dmd, '_L_HWH':_L_HWH, '_L_DHW':_L_DHW, '_L_BB_ba2':_L_BB_ba2 })
    
    # タンクユニットの補器消費電力
    _E_E_TU_aux_ba2_t = get_E_E_TU_aux_ba2_t(_CGS_name, time, _L_2dash_ba2)
    _E_E_TU_aux_HWH_t = get_E_E_TU_aux_HWH_t(_CGS_name, _L_HWH, _r_WS_ln_i) 
    _E_E_TU_aux_DHW_t = get_E_E_TU_aux_DHW_t(_CGS_name)
    _E_E_TU_aux_t = get_E_E_TU_aux_t(_E_E_TU_aux_DHW_t, _E_E_TU_aux_HWH_t, _E_E_TU_aux_ba2_t)
    
    #print({'_E_E_TU_aux_ba2': _E_E_TU_aux_ba2, '_E_E_TU_aux_HWH':_E_E_TU_aux_HWH, '_E_E_TU_aux_DHW':_E_E_TU_aux_DHW, '_E_E_TU_aux':_E_E_TU_aux})
    
    # 分担可能電力負荷
    _E_E_dmd_PU_t = get_E_E_dmd_PU_t(_E_E_dmd_t, _E_E_TU_aux_t)
    _E_E_PU_t = get_E_E_PU_t(_CGS_name, _E_E_dmd_PU_t)
    _E_E_PU = get_E_E_PU(_E_E_PU_t)
    
    # print({'_E_E_dmd_PU_t':_E_E_dmd_PU_t, '_E_E_PU_t':_E_E_PU_t, '_E_E_PU_t':_E_E_PU_t, '_E_E_PU':_E_E_PU})
    
    # 発電効率 and 排熱効率
    _e_H_PU = get_e_H_PU(_CGS_name, _E_E_PU, _L_DHW, _L_HWH)
    _e_E_PU = get_e_E_PU(_CGS_name, _E_E_PU, _L_DHW, _L_HWH)
    
    # print({'_e_H_PU':_e_H_PU, '_e_E_PU':_e_E_PU})
    
    # 燃料消費量
    _r_H_gen_PU_HVt = get_r_H_gen_PU_HVt(_CGS_name, _L_DHW, _L_HWH)
    _E_F_PU_HVt = get_E_F_PU_HVt(_CGS_name, _L_DHW, _L_HWH, _r_H_gen_PU_HVt, _e_H_PU)
    _E_E_gen_PU_EVt = get_E_E_gen_PU_EVt(_CGS_name, _E_E_PU, _L_DHW, _L_HWH)
    _E_F_PU_EVt = get_E_F_PU_EVt(_E_E_gen_PU_EVt, _e_E_PU)
    _E_F_PU = get_E_F_PU(_CGS_name, _E_F_PU_EVt, _E_F_PU_HVt)
    
    # print({'_r_H_gen_PU_HVt':_r_H_gen_PU_HVt, '_E_F_PU_HVt':_E_F_PU_HVt, '_E_E_gen_PU_EVt':_E_E_gen_PU_EVt, '_E_F_PU_EVt':_E_F_PU_EVt, '_E_F_PU':_E_F_PU})
    
    # 発電量
    _E_E_gen_PU = get_E_E_gen_PU(_E_F_PU, _e_E_PU)
    _E_E_gen_PU_t = get_E_E_gen_PU_t(_E_E_gen_PU, _E_E_PU_t, _E_E_PU)
    
    # print({'_E_E_gen_PU':_E_E_gen_PU, '_E_E_gen_PU_t':_E_E_gen_PU_t}) 
    
    # 排熱利用量
    _Q_PU_gen = get_Q_PU_gen(_E_F_PU, _e_H_PU)
    _Q_gen_DHW, _Q_gen_HWH = get_Q_gen_DHWandHWH(_CGS_name, _Q_PU_gen, _L_DHW, _L_HWH)
    
    print({'_Q_PU_gen':_Q_PU_gen, '_Q_gen_DHW':_Q_gen_DHW, '_Q_gen_HWH':_Q_gen_HWH}) 
    
    # 温水暖房時のBBの燃料消費量
    _L_BB_HWH_t = get_L_BB_HWH_t(_L_HWH_t, _Q_gen_HWH, _L_HWH)
    _Q_out_BB_HWH_t = get_Q_out_BB_HWH_t(_CGS_name, _L_BB_HWH_t)
    _E_F_BB_HWH = get_E_F_BB_HWH(_CGS_name, _Q_out_BB_HWH_t)
    
    ###### 式(6)、(7)が未完成（温水暖房使用時のみ使われる）
    print({'_L_BB_HWH_t':_L_BB_HWH_t, '_Q_out_BB_HWH_t':_Q_out_BB_HWH_t, '_E_F_BB_HWH':_E_F_BB_HWH})
    
    # BBが分担する給湯熱負荷
    _L_BB_DHW = get_L_BB_DHW( _L_DHW, _Q_gen_DHW )
    _L_BB_DHW_k, _L_BB_DHW_s, _L_BB_DHW_w, _L_BB_DHW_b1, _L_BB_DHW_b2, _L_BB_DHW_ba1, _L_BB_DHW_ba2 \
        = get_L_BB( _L_BB_DHW, _L_DHW, _L_2dash_k, _L_2dash_s, _L_2dash_w, _L_2dash_b1, _L_2dash_b2, _L_2dash_ba1, _L_2dash_ba2)
    
    print({'_L_BB_DHW':_L_BB_DHW, '_L_BB_DHW_k':_L_BB_DHW_k, '_L_BB_DHW_s':_L_BB_DHW_s, '_L_BB_DHW_w':_L_BB_DHW_w, '_L_BB_DHW_b1':_L_BB_DHW_b1,
          '_L_BB_DHW_b2':_L_BB_DHW_b2, '_L_BB_DHW_ba1':_L_BB_DHW_ba1, '_L_BB_DHW_ba2':_L_BB_DHW_ba2}) 
    
    # 給湯時のBBの燃料消費量
    _theta_ex_ave = np.mean(_theta_ex)
    _E_F_BB_DHW = get_E_hs( _CGS_name, _theta_ex_ave, _L_BB_DHW_k, _L_BB_DHW_s, _L_BB_DHW_w, _L_BB_DHW_b1, _L_BB_DHW_b2, _L_BB_DHW_ba1, _L_BB_DHW_ba2 )
    
    print({'_theta_ex_ave':_theta_ex_ave, '_E_F_BB_DHW':_E_F_BB_DHW})
       
    # 給湯時のBBの年間平均効率
    _e_BB_ave = get_e_BB_ave(_L_BB_DHW, _E_F_BB_DHW)
    
    print({'_e_BB_ave':_e_BB_ave})
    
    # 発電量
    _E_E_CG_gen_t = get_E_E_CG_gen_t( _E_E_gen_PU_t, _E_E_TU_aux_t )
    
    print({'_E_E_CG_gen_t':_E_E_CG_gen_t})
    
    # ガス消費量及び灯油消費量
    _E_CG = get_E_CG( _E_F_PU, _E_F_BB_DHW, _E_F_BB_HWH )
    
    print({'_E_CG':_E_CG})
    
    return _E_E_CG_gen_t, _E_CG
    

In [58]:
# 1/1のデータ(テストケース：CGA44：温水暖房なし)
_theta_ex = np.array([0.1,-0.4,-0.4,-0.9,-1.2,-1.3,-1.7,-1.4,-1.5,-1.0,-0.7,-0.1,0.6,0.0,-0.6,-0.8,-1.2,-1.0,-1.1,-0.9,-0.7,-0.4,-0.4,-0.3])
_E_E_H_t  = np.array([0, 0, 0, 0, 0, 0, 0, 4.421938, 2.056426, 1.228939, 1.013728, 0.560951, 0.324964, 0, 0, 1.425335, 1.183681, 1.182491, 0.605776, 1.581012, 1.13774, 1.09861, 0, 0])
_E_E_C_t  = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
_E_E_V_t  = np.array([0.048052, 0.048052, 0.048052, 0.048052, 0.048052, 0.048052, 0.048052, 0.049722, 0.062732, 0.048882, 0.049722, 0.048882, 0.061062, 0.048882, 0.048052, 0.048052, 0.049722, 0.073232, 0.073232, 0.048052, 0.048882, 0.072672, 0.073502, 0.060782])
_E_E_L_t  = np.array([0, 0, 0, 0, 0, 0, 0, 0.047338, 0.228004, 0.447402, 0.377723, 0.340251, 0.300125, 0.187957, 0, 0, 0.140643, 0.260091, 0.319217, 0.135328, 0.282022, 0.325786, 0.329829, 0.262385])
_E_E_AP_t = np.array([0.1812, 0.0717, 0.0795, 0.0795, 0.0717, 0.0795, 0.0795, 0.2774, 0.3702, 0.3272, 0.3134, 0.3157, 0.3705, 0.0717, 0.0795, 0.0795, 0.5453, 0.5187, 0.4476, 0.2056, 0.219, 0.3465, 0.3009, 0.1812])

# check
print(np.mean(_theta_ex), sum(_E_E_H_t), sum(_E_E_C_t), sum(_E_E_V_t), sum(_E_E_L_t), sum(_E_E_AP_t))
print(len(_theta_ex), len(_E_E_H_t), len(_E_E_C_t), len(_E_E_V_t), len(_E_E_L_t), len(_E_E_AP_t))

SpecData = {'time' : 10, '_CGS_name' : 'SOFC1', '_theta_ex':_theta_ex, '_E_E_H_t' : _E_E_H_t, '_E_E_C_t' :_E_E_C_t, '_E_E_V_t' : _E_E_V_t, '_E_E_L_t' : _E_E_L_t, '_E_E_AP_t' : _E_E_AP_t,
            '_L_HWH_t' : 0, '_L_2dash_k' : 27.711428, '_L_2dash_w' : 6.927857, '_L_2dash_s' : 36.145341, '_L_2dash_b1' : 0, '_L_2dash_b2' : 27.109006,
            '_L_2dash_ba1' : 0, '_L_2dash_ba2' : 7.608125, '_r_WS_ln_i' : [None, None, None, None, None]}

-0.7208333333333332 17.821591 0 1.302428 3.9841010000000003 5.6125
24 24 24 24 24 24


In [68]:
get_result(**SpecData)

{'_Q_PU_gen': 39.50997386062137, '_Q_gen_DHW': 28.553858109071065, '_Q_gen_HWH': 0}
{'_L_BB_HWH_t': 0, '_Q_out_BB_HWH_t': 0.0, '_E_F_BB_HWH': 0}
{'_L_BB_DHW': 69.33977389092894, '_L_BB_DHW_k': 19.628489743998436, '_L_BB_DHW_s': 25.602378019343725, '_L_BB_DHW_w': 4.907122435999609, '_L_BB_DHW_b1': 0.0, '_L_BB_DHW_b2': 19.201783691587174, '_L_BB_DHW_ba1': 0.0, '_L_BB_DHW_ba2': 7.608125}
{'_theta_ex_ave': -0.7208333333333332, '_E_F_BB_DHW': 91.96751672317184}
{'_e_BB_ave': 0.7539594017705853}
{'_E_E_CG_gen_t': array([0.21999283, 0.1146084 , 0.12211523, 0.12211523, 0.1146084 ,
       0.12211523, 0.12211523, 0.6565904 , 0.6565904 , 0.6565904 ,
       0.6565904 , 0.6565904 , 0.6565904 , 0.29629981, 0.12211523,
       0.6565904 , 0.6565904 , 0.6565904 , 0.6565904 , 0.6565904 ,
       0.6565904 , 0.6565904 , 0.6565904 , 0.48476759])}
{'_E_CG': 209.94198163295985}


(array([0.21999283, 0.1146084 , 0.12211523, 0.12211523, 0.1146084 ,
        0.12211523, 0.12211523, 0.6565904 , 0.6565904 , 0.6565904 ,
        0.6565904 , 0.6565904 , 0.6565904 , 0.29629981, 0.12211523,
        0.6565904 , 0.6565904 , 0.6565904 , 0.6565904 , 0.6565904 ,
        0.6565904 , 0.6565904 , 0.6565904 , 0.48476759]), 209.94198163295985)

In [ ]:
# 1/1のデータ(テストケース：CGA28：主居室に床暖房設置)
